In [4]:
import pandas as pd

train = pd.read_csv('./input/train.csv')
test  = pd.read_csv('./input/test.csv')


In [7]:
print(train.shape)
train.head()

(42000, 785)


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,pixel40,pixel41,pixel42,pixel43,pixel44,pixel45,pixel46,pixel47,pixel48,pixel49,...,pixel733,pixel734,pixel735,pixel736,pixel737,pixel738,pixel739,pixel740,pixel741,pixel742,pixel743,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
print(test.shape)
test.head()

(28000, 784)


,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,pixel40,pixel41,pixel42,pixel43,pixel44,pixel45,pixel46,pixel47,pixel48,pixel49,pixel50,...,pixel733,pixel734,pixel735,pixel736,pixel737,pixel738,pixel739,pixel740,pixel741,pixel742,pixel743,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
train_temp = train.copy()
Xtrain = train_temp.drop(columns=['label'], axis=1)
ytrain = train.label

In [25]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.impute   import SimpleImputer
# my_pipeline = make_pipeline(SimpleImputer(), RandomForestRegressor())

In [26]:
from sklearn.model_selection import cross_val_score
# scores = cross_val_score(my_pipeline, Xtrain, ytrain, cv=5, 
#                          scoring='neg_mean_absolute_error')
# print(scores)

/Users/i326864/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/i326864/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/i326864/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/i326864/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/i326864/anaconda3/lib/python3

[-0.54036905 -0.56295238 -0.5432381  -0.56877381 -0.53469048]


In [27]:
# print('Mean Absolute Error %2f' %(-1 * scores.mean()))

Mean Absolute Error 0.550005


In [31]:
#Evaluation Metrics
from sklearn.metrics import mean_squared_error, make_scorer

def rmse(predict, actual):
    score = mean_squared_error(ytrain, y_pred) ** 0.5
    return score
rmse_score = make_scorer(rmse)

def score(model):
    score = cross_val_score(model, Xtrain, ytrain, cv=5, 
                            scoring=rmse_score).mean()
    return score

scores = {}

In [32]:
from sklearn.metrics import mean_absolute_error, r2_score

#### Simple Linear Regression

In [35]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression(n_jobs=-1)
lr_model.fit(Xtrain, ytrain)

#accuracies = cross_val_score(estimator=lr_model,
                         #   X=Xtrain,
                         #   y=Ytrain,
                          #  cv=5,
                          #  verbose=1)
                
y_pred = lr_model.predict(Xtrain)

print('')
print('####### Linear Regression #######')
meanCV = score(lr_model)
print('Mean CV Score : %.4f' % meanCV)

mse = mean_squared_error(ytrain,y_pred)
mae = mean_absolute_error(ytrain, y_pred)
rmse = mean_squared_error(ytrain, y_pred)**0.5
r2 = r2_score(ytrain, y_pred)
scores.update({'OLS':[meanCV, mse, mae, rmse, r2]})

print('')
print('MSE(RSS)    : %0.4f ' % mse)
print('MAE         : %0.4f ' % mae)
print('RMSE        : %0.4f ' % rmse)
print('R2          : %0.4f ' % r2)


####### Linear Regression #######
Mean CV Score : 1.7755

MSE(RSS)    : 3.1523 
MAE         : 1.3920 
RMSE        : 1.7755 
R2          : 0.6220 


#### Lasso Regression

In [38]:
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Lasso

model_lasso = Lasso(random_state=42, alpha=0.00035)
lr_lasso = make_pipeline(RobustScaler(), model_lasso)
lr_lasso.fit(Xtrain, ytrain)

y_pred = lr_lasso.predict(Xtrain)

print('')
print('####### Lasso Regression #######')
meanCV = score(lr_lasso)
print('Mean CV Score : %.4f' % meanCV)

mse = mean_squared_error(ytrain,y_pred)
mae = mean_absolute_error(ytrain, y_pred)
rmse = mean_squared_error(ytrain, y_pred)**0.5
r2 = r2_score(ytrain, y_pred)
scores.update({'Lasso':[meanCV, mse, mae, rmse, r2]})

print('')
print('MSE(RSS)    : %0.4f ' % mse)
print('MAE         : %0.4f ' % mae)
print('RMSE        : %0.4f ' % rmse)
print('R2          : %0.4f ' % r2)


####### Lasso Regression #######
Mean CV Score : 1.7758

MSE(RSS)    : 3.1533 
MAE         : 1.3928 
RMSE        : 1.7758 
R2          : 0.6218 


#### Ridge Regression

In [39]:
from sklearn.linear_model import Ridge

lr_ridge = make_pipeline(RobustScaler(), 
                         Ridge(random_state=42,alpha=0.002))
lr_ridge.fit(Xtrain,ytrain)

y_pred = lr_ridge.predict(Xtrain)

print('')
print('####### Ridge Regression #######')
meanCV = score(lr_ridge)
print('Mean CV Score : %.4f' % meanCV)


mse = mean_squared_error(ytrain,y_pred)
mae = mean_absolute_error(ytrain, y_pred)
rmse = mean_squared_error(ytrain, y_pred)**0.5
r2 = r2_score(ytrain, y_pred)
scores.update({'Ridge':[meanCV, mse, mae, rmse, r2]})

print('')
print('MSE(RSS)    : %0.4f ' % mse)
print('MAE         : %0.4f ' % mae)
print('RMSE        : %0.4f ' % rmse)
print('R2          : %0.4f ' % r2)


####### Ridge Regression #######
Mean CV Score : 1.7755

MSE(RSS)    : 3.1523 
MAE         : 1.3920 
RMSE        : 1.7755 
R2          : 0.6220 


#### ElasticNet Regression

In [41]:
from sklearn.linear_model import ElasticNet

lr_elasticnet = make_pipeline(RobustScaler(),
                              ElasticNet(alpha=0.02, l1_ratio=0.7,random_state=42))
lr_elasticnet.fit(Xtrain,ytrain)

y_pred = lr_elasticnet.predict(Xtrain)

print('')
print('####### ElasticNet Regression #######')
meanCV = score(lr_elasticnet)
print('Mean CV Score : %.4f' % meanCV)

mse = mean_squared_error(ytrain,y_pred)
mae = mean_absolute_error(ytrain, y_pred)
rmse = mean_squared_error(ytrain, y_pred)**0.5
r2 = r2_score(ytrain, y_pred)
scores.update({'ElasticNet':[meanCV, mse, mae, rmse, r2]})

print('')
print('MSE(RSS)    : %0.4f ' % mse)
print('MAE         : %0.4f ' % mae)
print('RMSE        : %0.4f ' % rmse)
print('R2          : %0.4f ' % r2)


####### ElasticNet Regression #######
Mean CV Score : 1.8013

MSE(RSS)    : 3.2447 
MAE         : 1.4256 
RMSE        : 1.8013 
R2          : 0.6109 


#### KNN Regression

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knn = make_pipeline(RobustScaler(),KNeighborsRegressor())
knn.fit(Xtrain,ytrain)

y_pred = knn.predict(Xtrain)

print('')
print('####### KNN Regression #######')
meanCV = score(knn)
print('Mean CV Score : %.4f' % meanCV)

mse = mean_squared_error(ytrain,y_pred)
mae = mean_absolute_error(ytrain, y_pred)
rmse = mean_squared_error(ytrain, y_pred)**0.5
r2 = r2_score(ytrain, y_pred)
scores.update({'KNN':[meanCV, mse, mae, rmse, r2]})

print('')
print('MSE(RSS)    : %0.4f ' % mse)
print('MAE         : %0.4f ' % mae)
print('RMSE        : %0.4f ' % rmse)
print('R2          : %0.4f ' % r2)

#### GradientBoosting Regression

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

model_GBoost = GradientBoostingRegressor(n_estimators=3000, 
                                         learning_rate=0.05,
                                         max_depth=4, 
                                         max_features='sqrt',
                                         min_samples_leaf=15, 
                                         min_samples_split=10,
                                         loss='huber', 
                                         random_state =42)
model_GBoost.fit(Xtrain,ytrain)

y_pred = model_GBoost.predict(Xtrain)

print('')
print('####### GradientBoosting Regression #######')
meanCV = score(model_GBoost)
print('Mean CV Score : %.4f' % meanCV)

mse = mean_squared_error(ytrain,y_pred)
mae = mean_absolute_error(ytrain, y_pred)
rmse = mean_squared_error(ytrain, y_pred)**0.5
r2 = r2_score(ytrain, y_pred)
scores.update({'GradientBoosting':[meanCV, mse, mae, rmse, r2]})

print('')
print('MSE(RSS)    : %0.4f ' % mse)
print('MAE         : %0.4f ' % mae)
print('RMSE        : %0.4f ' % rmse)
print('R2          : %0.4f ' % r2)

#### RandomForest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(random_state=42)
forest_reg.fit(Xtrain, ytrain)

y_pred = forest_reg.predict(Xtrain)

print('')
print('####### RandomForest Regression #######')
meanCV = score(forest_reg)
print('Mean CV Score : %.4f' % meanCV)


mse = mean_squared_error(ytrain,y_pred)
mae = mean_absolute_error(ytrain, y_pred)
rmse = mean_squared_error(ytrain, y_pred)**0.5
r2 = r2_score(ytrain, y_pred)
scores.update({'RandomForest':[meanCV, mse, mae, rmse, r2]})

print('')
print('MSE(RSS)    : %0.4f ' % mse)
print('MAE         : %0.4f ' % mae)
print('RMSE        : %0.4f ' % rmse)
print('R2          : %0.4f ' % r2)

#### Grid Search for finding best params for RandomForest

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [70,100], 'max_features': [150]},
    {'bootstrap': [True], 'n_estimators': [70,100], 
     'max_features': [150]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', 
                           return_train_score=True)
grid_search.fit(Xtrain, ytrain)

y_pred = grid_search.predict(Xtrain) #???? missing

print('')
print('####### GridSearch RF Regression #######')
meanCV = score(grid_search)
print('Mean CV Score : %.4f' % meanCV)

mse = mean_squared_error(ytrain,y_pred)
mae = mean_absolute_error(ytrain, y_pred)
rmse = mean_squared_error(ytrain, y_pred)**0.5
r2 = r2_score(ytrain, y_pred)
scores.update({'GridSearchRF':[meanCV, mse, mae, rmse, r2]})

print('')
print('MSE(RSS)    : %0.4f ' % mse)
print('MAE         : %0.4f ' % mae)
print('RMSE        : %0.4f ' % rmse)
print('R2          : %0.4f ' % r2)

In [ ]:
grid_search.best_estimator_

In [ ]:
scores_list =[]
for k,v in scores.items():
    temp_lst =[]
    temp_lst.append(k)
    temp_lst.extend(v)
    scores_list.append(temp_lst)
    
scores_df = pd.DataFrame(scores_list, 
                         columns=['Model','CV_Mean_Score',
                                  'MSE(RSS)','MAE','RMSE',
                                  'R2Squared'])

scores_df.sort_values(['CV_Mean_Score'])

In [ ]:
_ = sns.scatterplot(x='Model',y='CV_Mean_Score',
                   data=scores_df,style='Model')

In [ ]:
Lasso_Predictions = lr_lasso.predict(test)

GBoost_Predictions = model_GBoost.predict(test)

KNN_Predictions = knn.predict(test)

GridSearch_Predictions = grid_search.best_estimator_.predict(test)

In [ ]:
submission=pd.read_csv('../input/sample_submission.csv')

submission['Label'] = Lasso_Predictions
submission.to_csv('../input/Lasso.csv',index=False)

submission['Label'] = GBoost_Predictions
submission.to_csv('../input/GBoost.csv',index=False)

submission['Label'] = KNN_Predictions
submission.to_csv('../input/KNN.csv',index=False)

submission['Label'] = GridSearch_Predictions
submission.to_csv('../input/GidSearch.csv',index=False)